# Optimal portfolios: 48 industry portfolios

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go
from scipy.optimize import minimize
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions

### Inputs

In [2]:
##### Inputs

# Read data and clean-up missing data (coded -99.99)
ff48 = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]

# Clean-up missings
for c in ff48.columns:
    ff48[c] = np.where(ff48[c]==-99.99, np.nan, ff48[c])
ff48 = ff48/100

ff48 = ff48.loc['1970-01':].copy()  # There is missing data prior to 1970

# Estimate inputs from historical data
MNS = ff48.mean().values
SDS = ff48.std()
C   = ff48.corr()
COV = np.diag(SDS) @ C @ np.diag(SDS)
COV = COV.to_numpy()

In [3]:
ff48.head(5)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-01,0.0083,-0.0281,-0.0276,-0.0135,-0.0699,-0.0795,-0.0574,-0.1139,-0.0690,-0.0509,...,-0.0849,-0.0762,-0.0768,-0.0573,-0.1186,-0.0753,-0.0888,-0.1094,-0.1082,-0.0359
1970-02,0.0948,0.0596,0.0386,0.0687,0.0028,0.0612,0.0822,0.0068,0.0028,0.0322,...,0.0637,0.1048,0.0181,0.0579,0.0543,0.1554,0.1035,0.0024,0.0918,-0.0481
1970-03,-0.1328,-0.0061,-0.0108,-0.0060,0.0142,-0.0280,-0.0252,-0.0289,-0.0096,-0.0109,...,-0.0237,-0.0334,-0.0528,-0.0098,-0.0752,-0.0120,-0.0025,-0.0095,-0.0059,0.0014
1970-04,-0.1764,-0.1057,-0.0867,-0.0941,-0.0282,-0.1990,-0.2182,-0.1517,-0.0792,-0.1435,...,-0.1069,-0.1196,-0.2056,-0.1013,-0.1827,-0.1090,-0.1527,-0.1870,-0.1258,-0.2720
1970-05,-0.1092,-0.0866,-0.0365,-0.0610,0.0354,-0.0774,-0.1315,-0.1755,-0.0824,-0.1019,...,-0.1025,-0.0706,-0.1159,-0.0949,-0.1219,-0.0649,-0.0872,-0.1503,-0.0945,-0.0655


## Efficient Frontier

In [4]:
def frontier(means, cov, target, Shorts):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if Shorts==1:
        # Constraint: short-sales allowed
        G = matrix(np.zeros((n,n)), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")        
    # Fully-invested constraint + E[r] = target
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, target], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

In [5]:
frontier(MNS, COV, 0.01, 1).round(3)

array([ 0.096,  0.089, -0.012,  0.022,  0.013, -0.043, -0.028, -0.027,
        0.225,  0.04 , -0.051,  0.083,  0.091, -0.109,  0.093,  0.024,
       -0.192, -0.088, -0.071,  0.052,  0.025, -0.043,  0.025, -0.05 ,
       -0.023,  0.091,  0.041,  0.049, -0.02 ,  0.088,  0.36 ,  0.202,
       -0.032, -0.062,  0.093, -0.018, -0.063,  0.105,  0.014,  0.104,
        0.04 ,  0.106, -0.056, -0.075, -0.019, -0.008,  0.026, -0.106])

In [6]:
frontier(MNS, COV, 0.01, 0).round(3)

array([0.02 , 0.128, 0.   , 0.029, 0.054, 0.   , 0.   , 0.   , 0.046,
       0.   , 0.   , 0.   , 0.118, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.052, 0.042,
       0.   , 0.   , 0.001, 0.372, 0.137, 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   ])

Frontier with short-selling

In [7]:
frontier_ss  = [frontier(MNS, COV, m, 1) for m in np.linspace(0.9*MNS.min(), 1.1*MNS.max(),50)]
expret_ss = [wgt @ MNS for wgt in frontier_ss]
sd_ss = [np.sqrt(wgt @ COV @ wgt) for wgt in frontier_ss]

Frontier without short-selling

In [8]:
frontier_noss  = [frontier(MNS, COV, m, 0) for m in np.linspace(MNS.min(), MNS.max(),50)]
expret_noss = [wgt @ MNS for wgt in frontier_noss]
sd_noss = [np.sqrt(wgt @ COV @ wgt) for wgt in frontier_noss]

Frontier without short-selling and with maximum position sizes

In [9]:
def constrained_frontier(means, cov, target, max_position):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: short-sales not allowed and maximum position limits
    G = matrix(np.vstack((-np.identity(n), np.identity(n))), tc="d")
    h = matrix(np.append(np.zeros(n), max_position*np.ones(n)), (2*n, 1), tc="d")        
    # Fully-invested constraint + E[r] = target
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, target], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

In [10]:
# what the new G is doing
np.vstack((-np.identity(4), np.identity(4)))

array([[-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [11]:
# what the new h is doing
np.append(np.zeros(4), 0.5*np.ones(4))

array([0. , 0. , 0. , 0. , 0.5, 0.5, 0.5, 0.5])

In [12]:
# The constrained frontier
frontier_maxlim  = [constrained_frontier(MNS, COV, m, 0.1) for m in np.linspace(MNS.min(), MNS.max(),50)]
expret_maxlim = [wgt @ MNS for wgt in frontier_maxlim]
sd_maxlim = [np.sqrt(wgt @ COV @ wgt) for wgt in frontier_maxlim]

## Global minimum variance

In [13]:
def gmv(means, cov, Shorts):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if Shorts==1:
        # Constraint: short-sales allowed
        G = matrix(np.zeros((n,n)), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts

GMV with shorting

In [14]:
wgts_gmv = gmv(MNS, COV, 1)
print(f'GMV weights\t\t {wgts_gmv.round(3)}')

# Portfolio expected return
gmv_expret = wgts_gmv @ MNS
print(f'GMV Expected Return:\t {gmv_expret: ,.4f}')

# Portfolio standard deviation
gmv_sd = np.sqrt(wgts_gmv @ COV @ wgts_gmv)
print(f'GMV Standard Deviation:\t {gmv_sd: ,.4f}')

GMV weights		 [ 0.1    0.086 -0.014  0.009 -0.008 -0.019 -0.039 -0.025  0.257  0.03
 -0.053  0.078  0.09  -0.11   0.072  0.019 -0.199 -0.088 -0.056  0.057
  0.026 -0.054  0.021 -0.061 -0.02   0.072  0.041  0.042 -0.019  0.081
  0.371  0.22  -0.004 -0.087  0.113 -0.031 -0.072  0.116  0.025  0.095
  0.046  0.116 -0.064 -0.069 -0.034  0.018  0.006 -0.078]
GMV Expected Return:	  0.0091
GMV Standard Deviation:	  0.0317


GMV without shorting

In [15]:
wgts_gmv_ssc = gmv(MNS, COV, 0)
print(f'GMV w/o shorting weights\t\t {wgts_gmv_ssc.round(3)}')

# Portfolio expected return
gmv_ssc_expret = wgts_gmv_ssc @ MNS
print(f'GMV w/o shorting Expected Return:\t {gmv_ssc_expret: ,.4f}')

# Portfolio standard deviation
gmv_ssc_sd = np.sqrt(wgts_gmv_ssc @ COV @ wgts_gmv_ssc)
print(f'GMV w/o shorting Standard Deviation:\t {gmv_ssc_sd: ,.4f}')

GMV w/o shorting weights		 [0.025 0.114 0.    0.    0.006 0.    0.    0.    0.143 0.    0.    0.
 0.088 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.006 0.048 0.    0.    0.    0.411 0.159 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
GMV w/o shorting Expected Return:	  0.0094
GMV w/o shorting Standard Deviation:	  0.0357


GMV without short-selling and with maximum position sizes

In [16]:
def constrained_gmv(means, cov, max_position):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: short-sales not allowed and maximum position limits
    G = matrix(np.vstack((-np.identity(n), np.identity(n))), tc="d")
    h = matrix(np.append(np.zeros(n), max_position*np.ones(n)), (2*n, 1), tc="d")   
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts    

In [17]:
wgts_gmv_maxlim = constrained_gmv(MNS, COV, 0.1)
print(f'GMV w/o shorting and max position limits: Weights\t\t {wgts_gmv_maxlim.round(3)}')

# Portfolio expected return
gmv_maxlim_expret = wgts_gmv_maxlim @ MNS
print(f'GMV w/o shorting and max position limits: Expected Return:\t {gmv_maxlim_expret: ,.4f}')

# Portfolio standard deviation
gmv_maxlim_sd = np.sqrt(wgts_gmv_maxlim @ COV @ wgts_gmv_maxlim)
print(f'GMV w/o shorting and max position limits: Standard Deviation:\t {gmv_maxlim_sd: ,.4f}')

GMV w/o shorting and max position limits: Weights		 [0.065 0.1   0.017 0.1   0.082 0.    0.    0.    0.1   0.    0.    0.025
 0.1   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.051 0.059 0.    0.    0.076 0.1   0.1   0.    0.    0.    0.
 0.    0.    0.001 0.    0.    0.024 0.    0.    0.    0.    0.    0.   ]
GMV w/o shorting and max position limits: Expected Return:	  0.0105
GMV w/o shorting and max position limits: Standard Deviation:	  0.0380


### The full picture

In [18]:
fig = go.Figure()

# Plot frontier with shorting
trace= go.Scatter(x=sd_ss, y=expret_ss,mode="lines", marker=dict(color="black"),name='Frontier with shorting')
fig.add_trace(trace)

# Plot GMV with shorting
trace= go.Scatter(x=[gmv_sd], y=[gmv_expret] ,
    mode="markers", marker=dict(size=10, color="black"), name='GMV with shorting')
fig.add_trace(trace)

# Plot frontier without shorting
trace= go.Scatter(x=sd_noss, y=expret_noss,mode="lines",marker=dict(color="blue"), name='Frontier without shorting')
fig.add_trace(trace)

# Plot GMV without shorting
trace= go.Scatter(x=[gmv_ssc_sd], y=[gmv_ssc_expret] ,
    mode="markers", marker=dict(size=10, color="blue"),name='GMV without shorting')
fig.add_trace(trace)

# Plot frontier without shorting and with max position sizes
trace= go.Scatter(x=sd_maxlim, y=expret_maxlim,mode="lines",marker=dict(color="purple"), name='Frontier without shorting + w/ max positions')
fig.add_trace(trace)

# Plot GMV without shorting  and with max position sizes
trace= go.Scatter(x=[gmv_maxlim_sd], y=[gmv_maxlim_expret] ,
    mode="markers", marker=dict(size=10, color="purple"),name='GMV without shorting + w/ max positions')
fig.add_trace(trace)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.4 * np.min(sd_ss), 1.1 * np.max(sd_noss)])
fig.update_yaxes(range=[0.7 * np.min(expret_ss), 1.1 * np.max(expret_ss)])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()